In [ ]:
%load_ext dotenv
%dotenv
OPENAI_API_KEY = %env OPENAI_API_KEY
%env DSP_CACHEDIR=./cache/library
%env DSP_NOTEBOOK_CACHEDIR=./cache/notebook

> This will use the repo's cache to not need to call LLMs

# Importing DSPy and DSPy Inspector

In [ ]:
from dsp.utils import deduplicate
import dspy
from dspy.teleprompt import BootstrapFewShot
from dspy.datasets import HotPotQA
from dspy_inspector import inspect

# Configuring the DSPy defaults

In [ ]:
lm = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=300)
rm = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(lm=lm, rm=rm)

dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]

# Creating a simple Multi-Hop RAG program

In [ ]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred): return False
    if not dspy.evaluate.answer_passage_match(example, pred): return False

    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]

    if max([len(h) for h in hops]) > 100: return False
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): return False

    return True

optimizer = BootstrapFewShot(metric=validate_context_and_answer_and_hops)

class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

class MultiHopRAG(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = [dspy.Retrieve(k=passages_per_hop) for _ in range(max_hops)]
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve[hop](query).passages
            context = deduplicate(context + passages)

        response = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=response.answer)

# Inspecting the RAG program

> It's the same as inspecting an instance of the RAG program.

In [ ]:
inspect(MultiHopRAG)

# Inspecting the uncompiled RAG program

> The created graph is bound to the created program's instance. Any subsequent calls to the program will update the graph.

Before any calls some information is empty, such as the parameters values.

In [ ]:
uncompiled_multiHopRAG = MultiHopRAG()
inspect(uncompiled_multiHopRAG)

After calling the program information is filled, check the parameters values.

In [ ]:
uncompiled_multiHopRAG("How many storeys are in the castle that David Gregory inherited?")

In [ ]:
uncompiled_multiHopRAG("When was the first FIFA World Cup held?")

In [ ]:
uncompiled_multiHopRAG("Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?")

# Inspecting the compiled RAG program

> Because we are creating a new instance the compilation won't affect the previous graph.

After compilation more information is filled, check the predictors demos.

In [ ]:
compiled_multiHopRAG = optimizer.compile(MultiHopRAG(), teacher=MultiHopRAG(passages_per_hop=2), trainset=trainset)
inspect(compiled_multiHopRAG)

In [ ]:
compiled_multiHopRAG("How many storeys are in the castle that David Gregory inherited?")

In [ ]:
compiled_multiHopRAG("When was the first FIFA World Cup held?")

In [ ]:
compiled_multiHopRAG("Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?")